# Test analytical gradients

Set up a toy example, with four different `Polyclonal` objects:
 - one with same variants for each concentration, and same with values fit
 - one with different variants for each concentration, and same with values fit
We do all these because handled slightly differently internally in `Polyclonal`.

In [1]:
import copy

import numpy

import pandas as pd

from polyclonal import Polyclonal

activity_wt_df = pd.DataFrame({"epitope": ["e1", "e2"], "activity": [2.0, 1.0]})

mut_escape_df = pd.DataFrame(
    {
        "mutation": [
            "M1C",
            "M1C",
            "G2A",
            "G2A",
            "A4K",
            "A4K",
            "A4L",
            "A4L",
            "A4Q",
            "A4Q",
        ],
        "epitope": ["e1", "e2", "e1", "e2", "e1", "e2", "e1", "e2", "e1", "e2"],
        "escape": [2.0, 0.0, 3.0, 0.0, 0.0, 2.5, 0.0, 1.5, 0.0, 3.5],
    }
)

polyclonal_sim = Polyclonal(activity_wt_df=activity_wt_df, mut_escape_df=mut_escape_df)

variants_df = pd.DataFrame.from_records(
    [
        ("AA", ""),
        ("AC", "M1C"),
        ("AG", "G2A"),
        ("AT", "A4K"),
        ("TA", "A4L"),
        ("GA", "A4Q"),
        ("CA", "M1C G2A"),
        ("CG", "M1C A4K"),
        ("TT", "M1C A4L"),
        ("GT", "M1C A4Q"),
        ("CC", "G2A A4K"),
        ("TC", "G2A A4L"),
        ("GG", "G2A A4Q"),
        ("CT", "M1C G2A A4K"),
        ("TG", "M1C G2A A4L"),
        ("GA", "M1C G2A A4Q"),
    ],
    columns=["barcode", "aa_substitutions"],
)

spatial_distances = pd.DataFrame(
    [(1, 2, 1.5), (1, 4, 7.5)],
    columns=["site_1", "site_2", "distance"],
)

escape_probs = polyclonal_sim.prob_escape(
    variants_df=variants_df, concentrations=[1.0, 2.0, 4.0]
)

data_to_fit = escape_probs.rename(columns={"predicted_prob_escape": "prob_escape"})

models = {}

models["same variants all concentration, not fit"] = Polyclonal(
    data_to_fit=data_to_fit,
    activity_wt_df=activity_wt_df,
    spatial_distances=spatial_distances,
    site_escape_df=pd.DataFrame.from_records(
        [("e1", 1, 1.0), ("e1", 4, 0.0), ("e2", 1, 0.0), ("e2", 4, 2.0)],
        columns=["epitope", "site", "escape"],
    ),
    data_mut_escape_overlap="fill_to_data",
)

# second model doesn't have the same mutations for all concentrations
models["different variants among concentrations, not fit"] = Polyclonal(
    data_to_fit=data_to_fit.head(len(data_to_fit) - 1),
    activity_wt_df=activity_wt_df,
    spatial_distances=spatial_distances,
    site_escape_df=pd.DataFrame.from_records(
        [("e1", 1, 1.0), ("e1", 4, 0.0), ("e2", 1, 0.0), ("e2", 4, 2.0)],
        columns=["epitope", "site", "escape"],
    ),
    data_mut_escape_overlap="fill_to_data",
)

models["non-one Hill coefficient, not fit"] = Polyclonal(
    data_to_fit=data_to_fit,
    activity_wt_df=activity_wt_df,
    hill_coefficient_df=pd.DataFrame(
        {"epitope": ["e1", "e2"], "hill_coefficient": [1.5, 0.9]}
    ),
    spatial_distances=spatial_distances,
    site_escape_df=pd.DataFrame.from_records(
        [("e1", 1, 1.0), ("e1", 4, 0.0), ("e2", 1, 0.0), ("e2", 4, 2.0)],
        columns=["epitope", "site", "escape"],
    ),
    data_mut_escape_overlap="fill_to_data",
)

models["non-zero non-neutralized frac, not fit"] = Polyclonal(
    data_to_fit=data_to_fit,
    activity_wt_df=activity_wt_df,
    non_neutralized_frac_df=pd.DataFrame(
        {"epitope": ["e1", "e2"], "non_neutralized_frac": [0.01, 0.1]}
    ),
    spatial_distances=spatial_distances,
    site_escape_df=pd.DataFrame.from_records(
        [("e1", 1, 1.0), ("e1", 4, 0.0), ("e2", 1, 0.0), ("e2", 4, 2.0)],
        columns=["epitope", "site", "escape"],
    ),
    data_mut_escape_overlap="fill_to_data",
)

# fit models
for name, model in list(models.items()):
    fit_name = name.replace("not fit", "fit")
    models[fit_name] = copy.deepcopy(model)
    _ = models[fit_name].fit()

# Gradient of $p_v\left(c\right)$

In [2]:
eps = 1e-8

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    p_vc, dpvc_dparams = model._compute_1d_pvs(
        params=model._params,
        one_binarymap=model._one_binarymap,
        binarymaps=model._binarymaps,
        cs=model._cs,
        calc_grad=True,
    )

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        p_vc_eps = model._compute_1d_pvs(
            params=model._params + eps_vec,
            one_binarymap=model._one_binarymap,
            binarymaps=model._binarymaps,
            cs=model._cs,
            calc_grad=False,
        )
        numerical_grad = (p_vc_eps - p_vc) / eps
        analytical_grad = dpvc_dparams[iparam].toarray()
        diff = numpy.sqrt(((numerical_grad - analytical_grad) ** 2).sum())
        mag = numpy.sqrt((analytical_grad**2).sum())
        if (diff > 1e-7) or (diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n" f"{analytical_grad=}\n{numerical_grad=}"
            )
    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of loss on $p_v\left(c\right)$

In [3]:
eps = 1e-7

delta = 2

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    loss, dloss = model._loss_dloss(params=model._params, delta=2)

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        loss_eps, _ = model._loss_dloss(params=model._params + eps_vec, delta=2)
        numerical_grad = (loss_eps - loss) / eps
        analytical_grad = dloss[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 5e-7) or (diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n" f"{analytical_grad=}\n{numerical_grad=}"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of escape regularization

In [4]:
eps = 1e-7

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    reg, dreg = model._reg_escape(params=model._params, weight=0.2, delta=0.5)

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        reg_eps, _ = model._reg_escape(
            params=model._params + eps_vec,
            weight=0.2,
            delta=0.5,
        )
        numerical_grad = (reg_eps - reg) / eps
        analytical_grad = dreg[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 1e-7) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n" f"{analytical_grad=}\n{numerical_grad=}"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of site spread regularization

In [5]:
eps = 1e-7

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    reg, dreg = model._reg_spread(
        params=model._params,
        weight=0.2,
    )

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        reg_eps, _ = model._reg_spread(
            params=model._params + eps_vec,
            weight=0.2,
        )
        numerical_grad = (reg_eps - reg) / eps
        analytical_grad = dreg[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 1e-7) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n"
                f"{analytical_grad=}\n{numerical_grad=}\n"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of site average absolute value (differentiable)

In [6]:
eps = 1e-7

epsilon = 0.1

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    s, ds = model._site_avg_abs_diff(model._params, epsilon)

    # numerical
    for iparam in range(len(model._params)):
        # add a small amount to params
        eps_vec = numpy.zeros(model._params.shape)
        eps_vec[iparam] = eps
        s_eps, _ = model._site_avg_abs_diff(model._params + eps_vec, epsilon)

        # figure out what beta value is being changed
        a_eps, n_eps, t_eps, beta_eps = model._a_n_t_beta_from_params(eps_vec)
        if (
            (a_eps == eps).sum() + (n_eps == eps).sum() + (t_eps == eps).sum() == 1
        ) and (beta_eps == eps).sum() == 0:
            # changing non-beta param should have no effect on s
            assert (s == s_eps).all()
            continue
        assert (a_eps == eps).sum() == (n_eps == eps).sum() == (t_eps == eps).sum() == 0
        assert (beta_eps == eps).sum() == 1
        i_mutation, i_epitope = numpy.where(beta_eps == eps)
        i_mutation = i_mutation[0]
        i_epitope == i_epitope[0]
        assert eps == beta_eps[i_mutation, i_epitope]

        site = (
            model._binarymaps.binary_sites[i_mutation]
            if model._one_binarymap
            else model._binarymaps[0].binary_sites[i_mutation]
        )
        siteindex = list(model._binary_sites).index(site)

        s_diff = (s_eps - s) / eps
        numerical_grad = s_diff[siteindex, i_epitope]
        analytical_grad = ds[i_mutation, i_epitope]

        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 1e-6) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n"
                f"{analytical_grad=}\n{numerical_grad=}\n"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of spatial regularization

In [7]:
eps = 1e-7

epsilon = 0.1
reg_spatial_weight = 0.1
reg_spatial2_weight = 0.01

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    reg, dreg = model._reg_spatial(
        params=model._params,
        reg_spatial_weight=reg_spatial_weight,
        reg_spatial2_weight=reg_spatial2_weight,
        epsilon=epsilon,
    )

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        reg_eps, _ = model._reg_spatial(
            params=model._params + eps_vec,
            reg_spatial_weight=reg_spatial_weight,
            reg_spatial2_weight=reg_spatial2_weight,
            epsilon=epsilon,
        )
        numerical_grad = (reg_eps - reg) / eps
        analytical_grad = dreg[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 1e-6) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n"
                f"{analytical_grad=}\n{numerical_grad=}\n"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of epitope uniqueness regularization

In [8]:
eps = 1e-7

epsilon = 0.1

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    reg, dreg = model._reg_uniqueness(
        params=model._params,
        weight=0.2,
        epsilon=epsilon,
    )

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        reg_eps, _ = model._reg_uniqueness(
            params=model._params + eps_vec,
            weight=0.2,
            epsilon=epsilon,
        )
        numerical_grad = (reg_eps - reg) / eps
        analytical_grad = dreg[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 2e-7) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n"
                f"{analytical_grad=}\n{numerical_grad=}\n"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of epitope uniqueness$^2$ regularization

In [9]:
eps = 1e-7

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    reg, dreg = model._reg_uniqueness2(
        params=model._params,
        weight=0.2,
    )

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        reg_eps, _ = model._reg_uniqueness2(
            params=model._params + eps_vec,
            weight=0.2,
        )
        numerical_grad = (reg_eps - reg) / eps
        analytical_grad = dreg[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 1e-5) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n"
                f"{analytical_grad=}\n{numerical_grad=}\n"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of Hill coefficient regularization

In [10]:
eps = 1e-7

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    reg, dreg = model._reg_hill_coefficient(
        params=model._params,
        weight=0.2,
    )

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        reg_eps, _ = model._reg_hill_coefficient(
            params=model._params + eps_vec,
            weight=0.2,
        )
        numerical_grad = (reg_eps - reg) / eps
        analytical_grad = dreg[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 1e-5) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n"
                f"{analytical_grad=}\n{numerical_grad=}\n"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check


## Gradient of non-neutralized fraction

In [11]:
eps = 1e-7

for name, model in models.items():
    print(f"\nModel: {name}")

    # analytical
    reg, dreg = model._reg_non_neutralized_frac(
        params=model._params,
        weight=0.2,
    )

    # numerical, compare to analytical
    for iparam in range(len(model._params)):
        eps_vec = numpy.zeros(len(model._params))
        eps_vec[iparam] = eps
        reg_eps, _ = model._reg_non_neutralized_frac(
            params=model._params + eps_vec,
            weight=0.2,
        )
        numerical_grad = (reg_eps - reg) / eps
        analytical_grad = dreg[iparam]
        diff = numpy.sqrt((numerical_grad - analytical_grad) ** 2)
        mag = numpy.sqrt(analytical_grad**2)
        if (diff > 1e-5) or (mag > 0 and diff / mag > 1e-3):
            raise ValueError(
                f"{iparam=}, {diff=}, {mag=}\n"
                f"{analytical_grad=}\n{numerical_grad=}\n"
            )

    print("All gradients check")


Model: same variants all concentration, not fit
All gradients check

Model: different variants among concentrations, not fit
All gradients check

Model: non-one Hill coefficient, not fit
All gradients check

Model: non-zero non-neutralized frac, not fit
All gradients check

Model: same variants all concentration, fit
All gradients check

Model: different variants among concentrations, fit
All gradients check

Model: non-one Hill coefficient, fit
All gradients check

Model: non-zero non-neutralized frac, fit
All gradients check
